In [27]:
from azure.identity import ClientSecretCredential
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_csv_agent
import pandas as pd
import configs.constants as const
import configs.deployments as dep
from langchain.callbacks import get_openai_callback

In [28]:
# authenticate to Azure
credentials = ClientSecretCredential(const.IQVIA_TENANT_ID, const.SERVICE_PRINCIPAL, const.SERVICE_PRINCIPAL_SECRET)
token = credentials.get_token(const.SCOPE_NON_INTERACTIVE)

In [29]:
# split excel sheets into csv files
df_0601 = pd.read_excel('C:\\Users\\u1128714\\Desktop\\langchain\\QC_summary_2023_06_1_subnational.xlsx', sheet_name=None)
for sheet_name in df_0601.keys():
    df_0601[sheet_name].to_csv(sheet_name + '_0601.csv', index=False)
df_0602 = pd.read_excel('C:\\Users\\u1128714\\Desktop\\langchain\\QC_summary_2023_06_2_subnational.xlsx', sheet_name=None)
for sheet_name in df_0602.keys():
    df_0602[sheet_name].to_csv(sheet_name + '_0602.csv', index=False)
df_ref_prod_0601 = pd.read_csv("C:\\Users\\u1128714\\Desktop\\langchain\\ref_prod_0601.csv")
df_ref_prod_0601.head()
df_ref_prod_0602 = pd.read_csv("C:\\Users\\u1128714\\Desktop\\langchain\\ref_prod_0602.csv")
df_ref_prod_0602.head()

,Unnamed: 0,PROD_CUSTOM_LVL1_DESC,COUNT(1)
0,0,ADBRY,2
1,1,CIBINQO,3
2,2,DUPIXENT,10
3,3,RINVOQ,4
4,4,TALTZ,8


In [30]:
llm = AzureChatOpenAI(
    openai_api_base=f"{const.OPENAI_API_BASE}/{const.OPENAI_API_TYPE}/{const.OPENAI_ACCOUNT_NAME}",
    openai_api_version=const.OPENAI_API_VERSION,
    openai_api_key=token.token,
    deployment_name=dep.GPT_35_TURBO,
    openai_api_type="azure_ad")

agent = create_csv_agent(llm, ['ref_prod_0601.csv', 'ref_prod_0602.csv'], verbose=True)

In [31]:
with get_openai_callback() as cb:
    response = agent.run("How different the PROD_CUSTOM_LVL1_DESC count is in the two files, just list the different product and count and changing percent?")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
response



> Entering new  chain...
Thought: I need to merge the two dataframes on the PROD_CUSTOM_LVL1_DESC column and then calculate the differences in COUNT(1) for each product.
Action: python_repl_ast
Action Input:
```
merged_df = df1.merge(df2, on='PROD_CUSTOM_LVL1_DESC', suffixes=('_df1', '_df2'))
merged_df['count_difference'] = merged_df['COUNT(1)_df2'] - merged_df['COUNT(1)_df1']
merged_df['changing_percent'] = merged_df['count_difference'] / merged_df['COUNT(1)_df1'] * 100
result_df = merged_df[['PROD_CUSTOM_LVL1_DESC', 'COUNT(1)_df1', 'COUNT(1)_df2', 'count_difference', 'changing_percent']]
result_df = result_df[result_df['count_difference'] != 0]
result_df = result_df.sort_values(by='count_difference', ascending=False)
print(result_df)
```
Observation:   PROD_CUSTOM_LVL1_DESC  COUNT(1)_df1  COUNT(1)_df2  count_difference  \
0                 ADBRY            12             2               -10   

   changing_percent  
0        -83.333333  

Thought:The only product with a count diffe

'ADBRY has a count difference of -10 and a changing percent of -83.33%.'